In [ ]:
import numpy as np
from fooof import FOOOF
from importlib import reload
import os
import mne
from matplotlib import pyplot as plt
# import own functions
from utils import find_folders
import fooof_ftg_fun
import json
import pandas as pd

In [ ]:
# Check the version of the tool
from fooof import __version__ as fooof_version
print('Current fooof version:', fooof_version)

from fooof.utils.reports import methods_report_info

In [ ]:
from fooof.utils.reports import methods_report_text

methods_report_text(fm)

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
fooof_epochs = pd.read_excel(os.path.join(
    project_path,
    'results', 'FOOOF_FIT',
    'fitting_epochs.xlsx'
))

fooof_epochs

In [ ]:
'''%matplotlib qt
fft_np = np.load(os.path.join(
        project_path,
            'data',
            'FFTs',
            'with_med_FTG',
            'Sub006_FFT.npy'
        )
    )

plt.pcolormesh(fft_np[0,:], cmap = 'viridis', vmin = -1, vmax = 7)
plt.ylim(5,100)'''

In [ ]:
reload(fooof_ftg_fun)

for i, filename in enumerate(fooof_epochs['Filename']):
    fft_np = np.load(os.path.join(
        project_path,
            'data',
            'FFTs',
            'with_med_FTG',
            filename
        )
    )

    SUBID = fooof_epochs.loc[i,'SubID']
    SIDE = fooof_epochs.loc[i,'SIDE']

    ON = fooof_epochs.loc[i,'Thres_On']
    OFF = fooof_epochs.loc[i,'Thres_Off']

    spectrum = np.mean(fft_np[SIDE,:,ON:OFF],1)

    PATH = os.path.join(project_path, 'results', 'FOOOF_FIT', 'with_med_DBSThres_knee')
    fm = fooof_ftg_fun.fit_fooof(SUBID, PATH, spectrum)

break


### Combine Jsons for each group

In [ ]:

json_dir = os.path.join(project_path, 'results', 'FOOOF_FIT', 'with_med_DBSThres_knee')

json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

data_list = []

for jk, file_name in enumerate(json_files):
    file_path = os.path.join(json_dir, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        if 'r_squared_' in data and 'error_' in data and 'aperiodic_params_' in data:
            # Extract "key1" and "key2"
            extracted_data = {
                'PerceptID': file_name[0:6],
                
                'Condition': 3, ############## CHANGE THIS

                'StimAmp': fooof_epochs.loc[jk, 'ThresAmp'],

                # Extract the elements from the "list_key" list
                'aperiodic_offset': data['aperiodic_params_'][0] if len(data['aperiodic_params_']) > 0 else None,
                'aperiodic_knee': data['aperiodic_params_'][1] if len(data['aperiodic_params_']) > 0 else None,
                'aperiodic_exponent': data['aperiodic_params_'][2] if len(data['aperiodic_params_']) > 1 else None,
                'r_squared': data['r_squared_'],
                'error': data['error_']
            }
            data_list.append(extracted_data)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data_list)

df

In [ ]:
df.to_csv(os.path.join(json_dir,'Foof_Aperiodics_DBSThres.csv'), index = False)

In [ ]:
#Load the two csvs
fooof_dbsoff = pd.read_csv(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Foof_Aperiodics_DBSOff.csv'
))

fooof_subthr = pd.read_csv(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Foof_Aperiodics_DBSSubThres.csv'
))

fooof_thres = pd.read_csv(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Foof_Aperiodics_DBSThres.csv'
))

all_pd = pd.concat([fooof_dbsoff, fooof_subthr, fooof_thres])

all_pd

In [ ]:
fooof_dbsoff['aperiodic_offset']

In [ ]:
%matplotlib qt
fig, axes = plt.subplots(1, 2, figsize=(9, 3))

axes[0].boxplot([fooof_dbsoff['aperiodic_offset'], fooof_subthr['aperiodic_offset'], fooof_thres['aperiodic_offset']], labels=['DBS Off', 'Pre-entrain', 'Entrainment'])
axes[0].plot([1, 2, 3], [fooof_dbsoff['aperiodic_offset'], fooof_subthr['aperiodic_offset'], fooof_thres['aperiodic_offset']], 'ko', markersize=4)
axes[0].plot([1, 2], [fooof_dbsoff['aperiodic_offset'], fooof_subthr['aperiodic_offset']], 'k:', alpha = 0.3)
axes[0].plot([2, 3], [fooof_subthr['aperiodic_offset'], fooof_thres['aperiodic_offset']], 'k:', alpha = 0.3)
axes[0].set_ylabel('Aperiodic Offset [a.u.]')

axes[1].boxplot([fooof_dbsoff['aperiodic_exponent'], fooof_subthr['aperiodic_exponent'], fooof_thres['aperiodic_exponent']], labels=['DBS Off', 'Pre-entrain', 'Entrainment'])
axes[1].plot([1, 2, 3], [fooof_dbsoff['aperiodic_exponent'], fooof_subthr['aperiodic_exponent'], fooof_thres['aperiodic_exponent']], 'ko', markersize=4)
axes[1].plot([1, 2], [fooof_dbsoff['aperiodic_exponent'], fooof_subthr['aperiodic_exponent']], 'k:', alpha = 0.3)
axes[1].plot([2, 3], [fooof_subthr['aperiodic_exponent'], fooof_thres['aperiodic_exponent']], 'k:', alpha = 0.3)
axes[1].set_ylabel('Aperiodic Exponent [a.u.]')

'''axes[2].boxplot([fooof_dbsoff['aperiodic_knee'], fooof_subthr['aperiodic_knee'], fooof_thres['aperiodic_knee']], labels=['DBS Off', 'Subthres', 'Thres'])
axes[2].plot([1, 2, 3], [fooof_dbsoff['aperiodic_knee'], fooof_subthr['aperiodic_knee'], fooof_thres['aperiodic_knee']], 'ko', markersize=4)
axes[2].plot([1, 2], [fooof_dbsoff['aperiodic_knee'], fooof_subthr['aperiodic_knee']], 'k:', alpha = 0.3)
axes[2].plot([2, 3], [fooof_subthr['aperiodic_knee'], fooof_thres['aperiodic_knee']], 'k:', alpha = 0.3)
axes[2].set_ylabel('Aperiodic Knee [a.u.]')
axes[2].set_ylim(-150, 900)'''


In [ ]:
plt.savefig(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Boxplots_Aperiodic'
), dpi = 400
)

plt.savefig(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Boxplots_Aperiodic.pdf'
))

In [ ]:
from scipy.stats import mannwhitneyu
statistic1, p_value1 = mannwhitneyu(fooof_with['aperiodic_offset'], fooof_without['aperiodic_offset'])
p_value1

In [ ]:
statistic2, p_value2 = mannwhitneyu(fooof_with['aperiodic_exponent'], fooof_without['aperiodic_exponent'])
p_value2